#Libraries and Data

In [1]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle
from statsmodels.tsa.stattools import adfuller

import xgboost as xgb
from datetime import datetime

from dataProcessing import DataProcessing
# Remove Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from ta import add_all_ta_features




from sklearn.model_selection import train_test_split

# Machine Learning
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# Evaluation
from sklearn.metrics import precision_score

# Reporting
import matplotlib.pyplot as plt

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
def load_and_setup_data(sybmol,input_data):
    df = pd.read_csv("../stock_historical_data/{}.csv".format(sybmol))
    df.set_index("Date", inplace=True)
    new_data = pd.DataFrame(input_data)
    new_data.set_index("Date", inplace=True)
    concatenated_df = pd.concat([df,new_data])
    return df

def fetch_stock_data(tickers):
    data = yfinance.download(tickers,period='1d')
    #print(data)
    return data
    

In [6]:
#symbols = ["OLECTRA.NS","LT.NS","CONCOR.NS","ELGIEQUIP.NS","IOC.NS","BEL.NS","TATAELXSI.NS","^NSEI"]
#symbols = ["OLECTRA.NS","CONCOR.NS","ELGIEQUIP.NS","IOC.NS","BEL.NS","TATAELXSI.NS","^NSEI","RELI","HDFCBANK.NS","TATAMOTORS.NS","SBIN.NS","TCS.NS","TITAN.NS","SUNPHARMA.BO","TECHM.NS", "ASIANPAINT.NS","TATACONSUM.NS"]
#symbols = ["^NSEI"]
symbols = ["^NSEI","RELI","HDFCBANK.NS","TATAMOTORS.NS","SBIN.NS","TCS.NS","OLECTRA.NS","CONCOR.NS","ELGIEQUIP.NS","IOC.NS","BEL.NS","TATAELXSI.NS","TITAN.NS","TECHM.NS", "ASIANPAINT.NS","TATACONSUM.NS"]
Icdate = 0
Iresult = 1
IisClassification = 2
Imodel = 3
Istock = 4
Iclose = 5
Iprob = 6
  # Add the tickers you want to fetch data for
loaded_models = {}
result = []

final_result = {}
final_target = {}
stock_data = fetch_stock_data(symbols)
print(stock_data)
finaldata = {}
current_date = datetime.now().strftime("%d-%m-%Y")

for symbol in symbols:
    stock_name = symbol
    if symbol == "^NSEI":
        stock_name = "NSEI"
    print(symbol)
    input_data = {
                    # "Open" : stock_data.head(1).Open[ticker].values[0],
                    "Open" : [stock_data.head(1).Open[symbol].values[0]],
                    "Close" : [stock_data.head(1).Close[symbol].values[0]],
                    "High" : [stock_data.head(1).High[symbol].values[0]],
                    "Low" : [stock_data.head(1).Low[symbol].values[0]], 
                    "Volume" : [stock_data.head(1).Volume[symbol].values[0]],
                    "Date":[np.datetime_as_string(stock_data.index, unit='D')[0]]
    }    #print(input_data)
    data = load_and_setup_data(symbol,input_data)
    data["Range"] = data["High"] / data["Low"] - 1
    data["Avg_Range"] = data["Range"].rolling(window=30).mean()
    data.dropna(inplace=True)
    # data.to_csv(f"../stock_historical_data/{symbol}.csv")
    #data = DataProcessing(symbol,data)
    # Specify Target
    # data.df.loc[data.df["Range"].shift(-1) > data.df["Avg_Range"], "TARGET"] = 1
    # data.df.loc[data.df["Range"].shift(-1) <= data.df["Avg_Range"], "TARGET"] = 0
    loaded_model = {}
    loaded_model_prophet = {}

    #feature_item = {}
    with open('../TrainedModel/ranage_xg/{}_model_phrophet.pkl'.format(symbol), 'rb') as f:
        loaded_model_prophet = pickle.load(f)
    with open('../TrainedModel/ranage_xg/{}_model_2.pkl'.format(symbol), 'rb') as f:
        loaded_model = pickle.load(f)
   # with open('../TrainedModel/xg/{}_features.txt'.format(symbol), 'rb') as f:
    #    feature_item = pickle.load(f)
    #final training
    loaded_models[symbol] = loaded_model
    data  = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
    data = data.dropna()
    data["ds"] = data.index
   # data["Date"] =  pd.to_datetime(data['Date'])
    data["y"] = data["Close"]


    forecast = loaded_model_prophet.predict(data)
    data = data.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
    
    #data = data.drop("index",axis=1)
    
    #get some variables
    prophet_variables = forecast.loc[:, ["trend", "weekly",  "multiplicative_terms","trend_upper","trend_lower"]]
    df_xgb = pd.concat([data, prophet_variables], axis = 1)
    non_stationaries = []
    for col in df_xgb.columns:
        if col != "multiplicative_terms" and col != "Date" and col != "ds" :
            #print(col)
            dftest = adfuller(df_xgb[col].values)
            p_value = dftest[1]
            t_test = dftest[0] < dftest[4]["1%"]
            if p_value > 0.05 or not t_test:
                non_stationaries.append(col)
    #print(f"Non-Stationary Features Found: {len(non_stationaries)}")
    
    df_stationary = df_xgb.copy()

    df_stationary = df_stationary.drop(columns=["Date","ds"])
    df_stationary[non_stationaries] = df_stationary[non_stationaries].pct_change()
    print(df_stationary)
    df_stationary = df_stationary[loaded_model.feature_names_in_]

   # score = loaded_model.predict(df_stationary)
    y_train_pred = loaded_model.predict_proba(df_stationary)[:,1]
    
    #print(y_train_pred)
        #df_xgb.head(1)
    #print(forecast)
   # final_target[symbol] = data.df["TARGET"]
   # df = data.df[feature_item]
   # finaldata[symbol]= df 
    stock_name_only = stock_name.replace(".NS","")
    train_yhat = loaded_model.predict(df_stationary[-1:])
    train_yhat_proba = loaded_model.predict_proba(df_stationary[-1:])
    greater = 0
    one = train_yhat_proba[0][0]
    two = train_yhat_proba[0][1]
    if one > two:
        greater = one*100
    else:
        greater = one*100
   # print(greater)
    #final_result[symbol] = [current_date,train_yhat[0],1,'xg',stock_name_only,stock_data.head(1).Close[symbol].values[0],round(greater,2)]
    result.append([current_date,train_yhat[0],1,'ranage_xg',stock_name_only,data.iloc[-1].Close,round(greater,2)])

print(result)
    

[*********************100%***********************]  16 of 16 completed
               Adj Close                                                   \
           ASIANPAINT.NS      BEL.NS   CONCOR.NS ELGIEQUIP.NS HDFCBANK.NS   
Date                                                                        
2023-07-21   3517.699951  124.949997  674.900024   561.349976     1675.75   

                                                                    ...  \
               IOC.NS   OLECTRA.NS  RELI     SBIN.NS TATACONSUM.NS  ...   
Date                                                                ...   
2023-07-21  99.349998  1275.150024  4.14  615.099976    852.349976  ...   

               Volume                                             \
           OLECTRA.NS  RELI   SBIN.NS TATACONSUM.NS TATAELXSI.NS   
Date                                                               
2023-07-21     743644  5300  21040281       1230534       190064   

                                              

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


          Open      High       Low     Close  Volume     Range  Avg_Range  \
0          NaN       NaN       NaN       NaN     NaN  0.037415        NaN   
1     0.036548  0.009033  0.021896 -0.006290     NaN  0.024358   0.009459   
2    -0.008221 -0.003974  0.004697  0.005436     NaN  0.015517  -0.000155   
3     0.005947  0.010030 -0.004112 -0.005796     NaN  0.029937  -0.011281   
4    -0.008291 -0.011178 -0.021063  0.011242     NaN  0.040338   0.031559   
...        ...       ...       ...       ...     ...       ...        ...   
3849  0.000000  0.000000 -0.000751 -0.000815     NaN  0.006987   0.003784   
3850 -0.000115  0.003040  0.000694  0.002036     NaN  0.009347   0.021999   
3851 -0.000090  0.001449  0.002461  0.007765     NaN  0.008328   0.026933   
3852  0.000000  0.000000  0.000000  0.000000     inf  0.008328   0.017059   
3853  0.000000  0.000000  0.000000  0.000000     0.0  0.008328   0.013794   

      volume_adi  volume_obv  volume_cmf  ...  momentum_kama  others_dr  \


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


         Open      High       Low     Close  Volume     Range  Avg_Range  \
0         NaN       NaN       NaN       NaN    6127       NaN   0.162658   
1   -0.005964  0.041626 -0.122934 -0.142857    9400  2.981806   0.146763   
2   -0.148000 -0.172862  0.003534  0.035211    2987 -0.833189   0.142694   
3    0.046948  0.053933  0.042253  0.015873    3673  0.262447   0.140126   
4    0.008969 -0.029851 -0.020270  0.004464    3147 -0.183448   0.133551   
..        ...       ...       ...       ...     ...       ...        ...   
577 -0.006316 -0.029586  0.006410 -0.016427   20600 -0.464969   0.066036   
578  0.004237 -0.016260 -0.006369 -0.014614   26200 -0.233210   0.060478   
579  0.012658 -0.008264 -0.053419 -0.061441   39800  1.443000   0.058929   
580 -0.037500 -0.031250 -0.006772  0.013544    8600 -0.319718   0.059499   
581 -0.028139  0.002150 -0.064591 -0.064588   21853  1.327110   0.060087   

     volume_adi  volume_obv  volume_cmf  ...  momentum_kama   others_dr  \
0           

...

          Open      High       Low     Close   Volume     Range  Avg_Range  \
0          NaN       NaN       NaN       NaN   416352  0.015940   0.044563   
1     0.002092  0.015690  0.013815  0.010422   827240  0.017820   0.043779   
2     0.007202  0.003707 -0.002463 -0.000938   509164  0.024116   0.043841   
3    -0.012333 -0.018418 -0.020648 -0.019554   440920  0.026449   0.044166   
4     0.002256  0.000836  0.002039  0.000425   418120  0.025217   0.043849   
...        ...       ...       ...       ...      ...       ...        ...   
3307 -0.008590 -0.020561 -0.013594 -0.007656  2088505  0.021442   0.019432   
3308 -0.004569  0.005388  0.008366  0.012466  1212566  0.018426   0.019507   
3309  0.009830  0.002401  0.007292 -0.001765  1274203  0.013481   0.019171   
3310 -0.005660  0.019589  0.000043  0.013671  3215610  0.033290   0.019325   
3311  0.021175  0.033644  0.025574  0.045437  6031344  0.041420   0.020136   

      volume_adi  volume_obv  volume_cmf  ...  momentum_kama  o

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


          Open      High       Low     Close   Volume     Range  Avg_Range  \
0          NaN       NaN       NaN       NaN   239100  0.073099   0.035629   
1    -0.054653 -0.051207 -0.007741 -0.011663   414590  0.026092   0.036126   
2    -0.010573 -0.014167 -0.014377 -0.016854   884210  0.026310   0.036630   
3    -0.026136 -0.016687 -0.005889  0.000672    72790  0.015162   0.036504   
4     0.013486  0.006317 -0.010923 -0.018073   119950  0.032857   0.037039   
...        ...       ...       ...       ...      ...       ...        ...   
3307 -0.011817 -0.012288 -0.012945 -0.000165   548838  0.023710   0.015546   
3308  0.001480  0.008274  0.015498  0.014777   772158  0.016427   0.015677   
3309  0.017329  0.004250  0.005157  0.004186  1025127  0.015510   0.015902   
3310  0.004988  0.003105  0.006573 -0.002407   750126  0.012011   0.015421   
3311 -0.007723  0.004088  0.000783  0.009180   832715  0.015353   0.015487   

      volume_adi  volume_obv  volume_cmf  ...  momentum_kama  o

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


          Open      High       Low     Close   Volume     Range  Avg_Range  \
0          NaN       NaN       NaN       NaN   155220  0.018821   0.038197   
1    -0.003141 -0.005204 -0.008536 -0.006763  1385110  0.022245   0.038375   
2    -0.011555 -0.006120  0.004224  0.000798   251740  0.011715   0.035214   
3    -0.002497 -0.010105 -0.014909 -0.014722   138290  0.016649   0.034301   
4    -0.002823 -0.001489  0.005405  0.003776   262530  0.009677   0.033494   
...        ...       ...       ...       ...      ...       ...        ...   
3307 -0.004201 -0.012903 -0.011782 -0.009121  1177214  0.023723   0.019147   
3308 -0.018419  0.001783  0.004562  0.017744  2167322  0.020890   0.019558   
3309  0.022942  0.005694  0.015804  0.003511  1435731  0.010730   0.019546   
3310  0.002426  0.001415 -0.004292 -0.006997   985928  0.016523   0.019548   
3311 -0.009504  0.004417 -0.001916  0.016899  1704921  0.022973   0.019803   

      volume_adi  volume_obv  volume_cmf  ...  momentum_kama  o

In [116]:

dataa = { "data":[]} 

for res in result:
    dataa["data"].append({ 
            "cdate": res[Icdate],
            "result": str(res[Iresult]),
            "pre_close" : str(res[Iclose]),            
            "isClassification": res[IisClassification],
            "model": res[Imodel],
            "stock": res[Istock],
            "prob": str(res[Iprob]),
    })
    
dataa

{'data': [{'cdate': '15-07-2023',
   'result': '0',
   'pre_close': '1373.6500244140625',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'OLECTRA',
   'prob': '51.47'},
  {'cdate': '15-07-2023',
   'result': '0',
   'pre_close': '688.7999877929688',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'CONCOR',
   'prob': '50.76'},
  {'cdate': '15-07-2023',
   'result': '0',
   'pre_close': '572.9500122070312',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'ELGIEQUIP',
   'prob': '54.26'},
  {'cdate': '15-07-2023',
   'result': '0',
   'pre_close': '96.5999984741211',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'IOC',
   'prob': '58.49'},
  {'cdate': '15-07-2023',
   'result': '0',
   'pre_close': '127.1500015258789',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'BEL',
   'prob': '55.74'},
  {'cdate': '15-07-2023',
   'result': '0',
   'pre_close': '7730.75',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'TATAELXSI',
   'prob': '50

In [117]:
import requests
import json

url = "https://website-development-kerala.com/api_214124524/ai_model_daily_runner.php"

response = requests.post(url, json=dataa)
print(response.status_code)
print(response)

200
<Response [200]>


In [118]:
final_target

{}

In [9]:
#for symbol in symbols:
   # retrained_model = loaded_models[symbol].fit(finaldata[symbol],final_target[symbol])
    #print(retrained_model)
    #with open('../TrainedModel/xg/{}_model_2.pkl'.format(symbol), 'wb') as f:
     #   pickle.dump(retrained_model, f)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=['error', 'logloss', 'logloss'], feature_types=None,
              gamma=6, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=25, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=1, ...)
XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eva